In [127]:
# most taken from https://github.com/achimkoh/midi-classification
##################
# Classification #
##################
composers = ("Bach", "Mozart", "Beethoven", "Debussy")

In [128]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn import svm, linear_model, naive_bayes, neural_network, neighbors, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import random, math
import numpy as np
import scipy.sparse as sp
from datetime import datetime
from collections import Counter
from itertools import combinations
from tqdm.notebook import *

In [129]:
with open('mxl-list.txt', 'r') as f:
    dataset = [piece.strip() for piece in f.readlines()]
    
    composer_datas = []
    for composer in composers:
        composer_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == composer]
        composer_datas.append(composer_data)
        
    '''bach_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'bach']
    beethoven_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'beethoven']
    debussy_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'debussy']
    scarlatti_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'scarlatti']
    victoria_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'victoria']'''

In [130]:
COMPOSERs = []
for composer, composer_data in zip(composers, composer_datas):
    with open(f'{composer}-chordsequence.txt', 'r') as f:
        COMPOSER = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
        COMPOSER = [(COMPOSER[i], composer_data[i]) for i in range(len(COMPOSER))]
        COMPOSERs.append(COMPOSER)
    
'''with open('bach-chordsequence.txt', 'r') as f:
    BACH = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    BACH = [(BACH[i], bach_data[i]) for i in range(len(BACH))]
with open('beethoven-chordsequence.txt', 'r') as f:
    BEETHOVEN = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    BEETHOVEN = [(BEETHOVEN[i], beethoven_data[i]) for i in range(len(BEETHOVEN))]
with open('debussy-chordsequence.txt', 'r') as f:
    DEBUSSY = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    DEBUSSY = [(DEBUSSY[i], debussy_data[i]) for i in range(len(DEBUSSY))]
with open('scarlatti-chordsequence.txt', 'r') as f:
    SCARLATTI = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    SCARLATTI = [(SCARLATTI[i], scarlatti_data[i]) for i in range(len(SCARLATTI))]
with open('victoria-chordsequence.txt', 'r') as f:
    VICTORIA = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    VICTORIA = [(VICTORIA[i], victoria_data[i]) for i in range(len(VICTORIA))]'''

"with open('bach-chordsequence.txt', 'r') as f:\n    BACH = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]\n    BACH = [(BACH[i], bach_data[i]) for i in range(len(BACH))]\nwith open('beethoven-chordsequence.txt', 'r') as f:\n    BEETHOVEN = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]\n    BEETHOVEN = [(BEETHOVEN[i], beethoven_data[i]) for i in range(len(BEETHOVEN))]\nwith open('debussy-chordsequence.txt', 'r') as f:\n    DEBUSSY = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]\n    DEBUSSY = [(DEBUSSY[i], debussy_data[i]) for i in range(len(DEBUSSY))]\nwith open('scarlatti-chordsequence.txt', 'r') as f:\n    SCARLATTI = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]\n    SCARLATTI = [(SCARLATTI[i], scarlatti_data[i]) for i in range(len(SCARLATTI))]\nwith open('victoria-chordsequence.txt', 'r') as f:\n    VICTORIA = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]\n  

In [131]:
def find_ngrams(input_list, N=4):
    return [' '.join(input_list[i:i+N]) for i in range(len(input_list)-N+1)]

def ngrams_by_composer(composer): 
    for i in range(1,5):
        ngrams = []
        for piece in composer:
            ngrams += find_ngrams(piece[0].split(' '), i)
        print(len(ngrams), '{}-grams total;'.format(str(i)), len(set(ngrams)), 'unique')
    print('-')

def show_ngrams(composer_data, composer_name):
    print(composer_name, ':', len(composer_data), 'pieces')
    ngrams_by_composer(composer_data)

In [132]:
for COMPOSER, composer in zip(COMPOSERs, composers):
    show_ngrams(COMPOSER, composer)
show_ngrams(sum(COMPOSERs, []), 'all composers')

'''show_ngrams(BACH,'bach')
show_ngrams(BEETHOVEN,'beethoven')
show_ngrams(DEBUSSY,'debussy')
show_ngrams(SCARLATTI,'scarlatti')
show_ngrams(VICTORIA, 'victoria')
show_ngrams(BACH+BEETHOVEN+DEBUSSY+SCARLATTI+VICTORIA, 'all composers')'''

Bach : 50 pieces
23473 1-grams total; 175 unique
23423 2-grams total; 5064 unique
23373 3-grams total; 12651 unique
23323 4-grams total; 16163 unique
-
Mozart : 50 pieces
37893 1-grams total; 177 unique
37843 2-grams total; 5668 unique
37793 3-grams total; 15811 unique
37743 4-grams total; 21892 unique
-
Beethoven : 50 pieces
39567 1-grams total; 179 unique
39517 2-grams total; 7082 unique
39467 3-grams total; 19873 unique
39417 4-grams total; 26545 unique
-
Debussy : 50 pieces
22281 1-grams total; 175 unique
22231 2-grams total; 5794 unique
22181 3-grams total; 13530 unique
22131 4-grams total; 17142 unique
-
all composers : 200 pieces
123214 1-grams total; 179 unique
123014 2-grams total; 12371 unique
122814 3-grams total; 51552 unique
122614 4-grams total; 76962 unique
-


"show_ngrams(BACH,'bach')\nshow_ngrams(BEETHOVEN,'beethoven')\nshow_ngrams(DEBUSSY,'debussy')\nshow_ngrams(SCARLATTI,'scarlatti')\nshow_ngrams(VICTORIA, 'victoria')\nshow_ngrams(BACH+BEETHOVEN+DEBUSSY+SCARLATTI+VICTORIA, 'all composers')"

In [133]:
def build_Xy(composers, size=1):
    if size >= 1: # use every rows
        indices = [range(len(composer)) for composer in composers]
    else:
        indices = [random.sample(range(len(composer)), math.ceil(size*len(composer))) for composer in composers]

    y = []
    for i in range(len(composers)):
        y += [i for n in range(len(indices[i]))]
    
    X = []
    for i in range(len(composers)):
        X += [composers[i][j] for j in indices[i]]
    
    return X, np.array(y, dtype='int16')

In [213]:
def crossvalidate(X_tuple, y, classifiers, vectorizer, NGRAMRANGE, K=10, set_=False):    
    for clf in classifiers:
        clf.cm_sum = np.zeros([len(set(y)),len(set(y))], dtype='int16') \
                     if set_ else np.zeros([len(composers), len(composers)], dtype='int16')
        clf.accuracies, clf.fones, clf.misclassified, clf.runningtime = [], [], [], []
        clf.fones_micro, clf.fones_macro = [], []
        clf.name = str(clf).split('(')[0]

    X = np.array([piece[0] for piece in X_tuple])
    
    import pickle #
    ids = [int(piece[1][:-4].split('_')[1]) for piece in X_tuple] #
    with open('indices.pickle', 'rb') as file:
        df = pickle.load(file)
    indices = np.asarray([df.loc[id_] for id_ in ids])
    
    filenames = np.array([piece[1] for piece in X_tuple])
    kf = KFold(n_splits=min(K,len(y)), shuffle=True)
    for train_index, test_index in tqdm(kf.split(y), unit='fold', total=10, leave=False):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
        vct = vectorizer.set_params(lowercase=False, token_pattern=u"(?u)\\b\\w+\\b", ngram_range=NGRAMRANGE)
        X_train_tfidf = vct.fit_transform(X_train)
#         X_test_tfidf = tfidf.transform(X_test)  # i think this computes tf-idf values using the whole test set, but i want each piece to be treated separately
        X_test_tfidf = sp.vstack([vct.transform(np.array([piece])) for piece in X_test])
    
        #print(type(X_train_tfidf), X_train_tfidf.shape)
        X_train_tfidf = np.array([list(ai) + list(bi) for ai, bi in zip(X_train_tfidf.toarray(), indices)])
        X_test_tfidf = np.array([list(ai) + list(bi) for ai, bi in zip(X_test_tfidf.toarray(), indices)])
        #print(X_train_tfidf)
        for clf in tqdm(classifiers, unit='classifier', leave=False):
            t = datetime.now()
            clf.fit(X_train_tfidf, y_train)
            y_pred = clf.predict(X_test_tfidf)
            clf.runningtime.append((datetime.now()-t).total_seconds())
            clf.cm_sum += confusion_matrix(y_test, y_pred) #, labels=list(set(y_test)) if set_ else range(1, len(composers)+1)
            clf.misclassified.append(test_index[np.where(y_test != y_pred)]) # http://stackoverflow.com/a/25570632
            clf.accuracies.append(accuracy_score(y_test, y_pred))
            clf.fones.append(f1_score(y_test, y_pred, average='weighted'))
            clf.fones_micro.append(f1_score(y_test, y_pred, average='micro'))
            clf.fones_macro.append(f1_score(y_test, y_pred, average='macro'))

    result = dict()
    for clf in classifiers:
        clf.misclassified = np.sort(np.hstack(clf.misclassified))
        result[clf.name] = [clf.cm_sum, clf.accuracies, clf.fones, clf.misclassified, filenames[clf.misclassified], clf.runningtime, clf.fones_micro, clf.fones_macro]
    return result

In [205]:
def benchmark_classifiers(composers, NGRAMRANGES, classifiers, vectorizer, n=1, retrieve_title=True, set_=False):
    misclassified_list = []
    for NGRAMRANGE in tqdm(NGRAMRANGES, unit='NGRAMRANGE', leave=False):
        print('n-gram range', NGRAMRANGE)
        X, y = build_Xy(composers, size=n)
        cv_result = crossvalidate(X, y, classifiers, vectorizer, NGRAMRANGE, set_=set_)
        for clf, results in cv_result.items():
            print(clf)
            cm = results[0]
            print(cm)
            acc = results[1] # using two different f-measures, don't need this
#             print('accuracy', round(np.mean(acc)*100,2), '({})'.format(round(np.std(acc, ddof=1)*100,2)))
            fones = results[2] # weighted average, don't need this
#             print('f1', round(np.mean(fones)*100,2), '({})'.format(round(np.std(fones, ddof=1)*100,2)), fones)
            misclassified = results[3]
            misclassified_filenames = results[4]
            misclassified_list += list(misclassified_filenames)
#             print('misclassified',[(misclassified[i], misclassified_filenames[i]) for i in range(len(misclassified))])
            runningtime = results[5]
#             print('running time', np.sum(runningtime))
            fones_micro = results[6]
            fones_macro = results[7]
            print('micro-averaged f-score (std) & macro-averaged f-score (std)')
            print(round(np.mean(fones_micro),4), '({})'.format(round(np.std(fones_micro, ddof=1),4)), '&', round(np.mean(fones_macro),4), '({})'.format(round(np.std(fones_macro, ddof=1),4)))
    print('-----')
    return misclassified_list

In [228]:
'''COMPOSERS = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]'''
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
NGRAMRANGES = [(1,2), (1,3)] #[(1,1),(2,2),(3,3),(4,4),(1,2),(3,4),(1,4)]
CLASSIFIERS = [
    svm.LinearSVC(),#penalty='l2', C=5, loss='hinge'),
    linear_model.LogisticRegression(), #penalty='l2', C=100, tol=1, multi_class='multinomial', solver='sag'),
    neighbors.KNeighborsClassifier(),#weights='distance'),
    #naive_bayes.MultinomialNB(alpha=0.00001, fit_prior=False),
    neural_network.MLPClassifier(),#solver='lbfgs', hidden_layer_sizes=(100,)),
    RandomForestClassifier(),#n_estimators=100, random_state=42),
    GradientBoostingClassifier(),#n_estimators=100, max_leaf_nodes=4, max_depth=None, random_state=2,
                   #min_samples_split=5),
]

In [229]:
VECTORIZER = TfidfVectorizer(sublinear_tf=True)
benchmark_classifiers(COMPOSERs,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

n-gram range (1, 2)


LinearSVC
[[40  4  5  1]
 [25 16  8  1]
 [13 11 19  7]
 [ 7  0 11 32]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.535 (0.118) & 0.5011 (0.1383)
LogisticRegression
[[32  6 11  1]
 [31  8  9  2]
 [10 15 16  9]
 [10  4 20 16]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.36 (0.0738) & 0.3366 (0.0734)
KNeighborsClassifier
[[32  9  9  0]
 [42  0  6  2]
 [34  6  2  8]
 [36 14  0  0]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.17 (0.0715) & 0.0963 (0.0412)
MLPClassifier
[[38  5  7  0]
 [22 20  8  0]
 [11 13 22  4]
 [ 5  1  8 36]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.58 (0.1183) & 0.5492 (0.1315)
RandomForestClassifier
[[36  6  7  1]
 [12 26 11  1]
 [ 5 12 30  3]
 [ 6  1  3 40]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.66 (0.1197) & 0.6561 (0.1247)
GradientBoostingClassifier
[[26 12 11  1]
 [22 21  6  1]
 [11  8 26  5]
 [ 6  0  4 40]]
micro-averaged f-score (std) & macro-averaged f-score (std)


LinearSVC
[[35  5 10  0]
 [27 15  5  3]
 [13 11 17  9]
 [ 9  0 13 28]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.475 (0.1339) & 0.4328 (0.1435)
LogisticRegression
[[32  4 13  1]
 [29  8  9  4]
 [15 18 10  7]
 [ 9  7 19 15]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.325 (0.0979) & 0.2989 (0.0966)
KNeighborsClassifier
[[29 11 10  0]
 [40  1  5  4]
 [34  7  3  6]
 [37 13  0  0]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.165 (0.058) & 0.1003 (0.0311)
MLPClassifier
[[37  5  8  0]
 [22 19  9  0]
 [14 12 22  2]
 [ 6  0 10 34]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.56 (0.0843) & 0.5116 (0.0776)
RandomForestClassifier
[[31  9  9  1]
 [11 29  9  1]
 [ 4 13 28  5]
 [ 4  0  1 45]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.665 (0.1415) & 0.6354 (0.1626)
GradientBoostingClassifier
[[27 10 11  2]
 [25 18  4  3]
 [14 11 20  5]
 [ 5  0  3 42]]
micro-averaged f-score (std) & macro-averaged f-score (st

['Bach_007.mxl',
 'Bach_013.mxl',
 'Bach_017.mxl',
 'Bach_019.mxl',
 'Bach_020.mxl',
 'Bach_033.mxl',
 'Bach_038.mxl',
 'Bach_041.mxl',
 'Bach_045.mxl',
 'Bach_047.mxl',
 'Mozart_050.mxl',
 'Mozart_051.mxl',
 'Mozart_052.mxl',
 'Mozart_054.mxl',
 'Mozart_055.mxl',
 'Mozart_057.mxl',
 'Mozart_058.mxl',
 'Mozart_059.mxl',
 'Mozart_060.mxl',
 'Mozart_061.mxl',
 'Mozart_062.mxl',
 'Mozart_063.mxl',
 'Mozart_065.mxl',
 'Mozart_066.mxl',
 'Mozart_067.mxl',
 'Mozart_069.mxl',
 'Mozart_071.mxl',
 'Mozart_072.mxl',
 'Mozart_073.mxl',
 'Mozart_074.mxl',
 'Mozart_075.mxl',
 'Mozart_076.mxl',
 'Mozart_079.mxl',
 'Mozart_080.mxl',
 'Mozart_081.mxl',
 'Mozart_082.mxl',
 'Mozart_083.mxl',
 'Mozart_084.mxl',
 'Mozart_085.mxl',
 'Mozart_086.mxl',
 'Mozart_090.mxl',
 'Mozart_093.mxl',
 'Mozart_095.mxl',
 'Mozart_099.mxl',
 'Beethoven_101.mxl',
 'Beethoven_102.mxl',
 'Beethoven_103.mxl',
 'Beethoven_104.mxl',
 'Beethoven_107.mxl',
 'Beethoven_108.mxl',
 'Beethoven_110.mxl',
 'Beethoven_112.mxl',
 'Beetho

In [ ]:
# Compare different methods of vectorizing

In [ ]:
VECTORIZER = CountVectorizer(binary=True)
benchmark_classifiers(COMPOSERs,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

In [ ]:
VECTORIZER = CountVectorizer()
benchmark_classifiers(COMPOSERs,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

In [ ]:
# test pairwise classification 

In [214]:
if True:
    NGRAMRANGES = [(1,2)]
    VECTORIZER = TfidfVectorizer(sublinear_tf=True)
    for indices in tqdm(list(combinations(range(len(composers)),2))):
        print(f'comparing {composers[indices[0]]} and {composers[indices[1]]} (composers {indices[0]} and {indices[1]})',) # COMPOSERs = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]
        benchmark_classifiers([COMPOSERs[i] for i in indices],NGRAMRANGES,CLASSIFIERS,VECTORIZER, set_=True)

comparing Bach and Mozart (composers 0 and 1)


n-gram range (1, 2)


LinearSVC
[[48  2]
 [28 22]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.7 (0.1764) & 0.6681 (0.1842)
LogisticRegression
[[46  4]
 [38 12]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.58 (0.1814) & 0.5252 (0.16)
KNeighborsClassifier
[[50  0]
 [40 10]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.6 (0.1944) & 0.5277 (0.1612)
MLPClassifier
[[48  2]
 [33 17]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.65 (0.2321) & 0.6141 (0.2321)
RandomForestClassifier
[[40 10]
 [18 32]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.72 (0.1619) & 0.7055 (0.171)
GradientBoostingClassifier
[[40 10]
 [19 31]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.71 (0.1524) & 0.6956 (0.1561)
-----
comparing Bach and Beethoven (composers 0 and 2)


n-gram range (1, 2)


C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

LinearSVC
[[36 14]
 [17 33]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.69 (0.1524) & 0.6722 (0.1612)
LogisticRegression
[[29 21]
 [18 32]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.61 (0.0994) & 0.5418 (0.0908)
KNeighborsClassifier
[[38 12]
 [47  3]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.41 (0.1287) & 0.321 (0.1093)
MLPClassifier
[[41  9]
 [42  8]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.49 (0.1449) & 0.4013 (0.1418)
RandomForestClassifier
[[36 14]
 [17 33]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.69 (0.1792) & 0.6793 (0.1854)
GradientBoostingClassifier
[[34 16]
 [25 25]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.59 (0.0994) & 0.5794 (0.0929)
-----
comparing Mozart and Debussy (composers 1 and 3)


n-gram range (1, 2)


LinearSVC
[[49  1]
 [19 31]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.8 (0.1491) & 0.7928 (0.1546)
LogisticRegression
[[38 12]
 [45  5]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.43 (0.0949) & 0.3348 (0.0889)
KNeighborsClassifier
[[38 12]
 [39 11]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.49 (0.0738) & 0.4352 (0.086)
MLPClassifier
[[48  2]
 [42  8]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.56 (0.2119) & 0.4766 (0.2226)
RandomForestClassifier
[[43  7]
 [ 8 42]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.85 (0.085) & 0.8388 (0.0926)
GradientBoostingClassifier
[[46  4]
 [ 8 42]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.88 (0.1398) & 0.8673 (0.1454)
-----
comparing Beethoven and Debussy (composers 2 and 3)


n-gram range (1, 2)


LinearSVC
[[47  3]
 [18 32]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.79 (0.137) & 0.784 (0.1361)
LogisticRegression
[[32 18]
 [24 26]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.58 (0.1229) & 0.5398 (0.1576)
KNeighborsClassifier
[[50  0]
 [46  4]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.54 (0.143) & 0.4039 (0.1274)
MLPClassifier
[[47  3]
 [30 20]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.67 (0.1252) & 0.6243 (0.1521)
RandomForestClassifier
[[46  4]
 [11 39]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.85 (0.108) & 0.8404 (0.118)
GradientBoostingClassifier
[[41  9]
 [ 9 41]]
micro-averaged f-score (std) & macro-averaged f-score (std)
0.82 (0.1751) & 0.8164 (0.1761)
-----



In [ ]:
# Identify the often-misclassified files
# do the experiment 100 times with the best classifier, SVM
# then find pieces that are misclassified more than 50% of the time
'''COMPOSERS = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]'''
CLASSIFIERS = [svm.LinearSVC(penalty='l2', C=5, loss='hinge')] # 
appendix = []
for i in trange(100):
    appendix += benchmark_classifiers(COMPOSERs,NGRAMRANGES,CLASSIFIERS,VECTORIZER)
Counter(appendix).most_common()

In [ ]:
# Use both chord sequences and duration data

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
with open('Bach-durations.txt', 'r') as f:
    BA = [line.strip() for line in f.readlines()]
with open('Mozart-durations.txt', 'r') as f:
    MO = [line.strip() for line in f.readlines()]
with open('Beethoven-durations.txt', 'r') as f:
    BE = [line.strip() for line in f.readlines()]
with open('Debussy-durations.txt', 'r') as f:
    DE = [line.strip() for line in f.readlines()]
'''with open('victoria-durations.txt', 'r') as f:
    VD = [line.strip() for line in f.readlines()]'''
    
#BD2_TYPELENGTH = [piece.split(';') for piece in BD2]
BA_TYPELENGTH = [piece.split(';') for piece in BA]
MO_TYPELENGTH = [piece.split(';') for piece in MO]
BE_TYPELENGTH = [piece.split(';') for piece in BE]
DE_TYPELENGTH = [piece.split(';') for piece in DE]

typelengths = list(set(flatten(BA_TYPELENGTH + MO_TYPELENGTH + BE_TYPELENGTH + DE_TYPELENGTH)))
typelength_dict = {typelengths[i]: str(i+300) for i in range(len(typelengths))}
BA_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in BA_TYPELENGTH]
MO_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in MO_TYPELENGTH]
BE_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in BE_TYPELENGTH]
DE_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in DE_TYPELENGTH]
#VD_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in VD_TYPELENGTH]

In [ ]:
# print most common duration by composer, regardless of element type(chord/note/rest)

# BD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in BD]
# SD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in SD]
# BD2_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in BD2]
# DD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in DD]
# VD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in VD]
# lengths = list(set(flatten(BD2_LENGTHONLY+BD_LENGTHONLY+DD_LENGTHONLY+SD_LENGTHONLY+VD_LENGTHONLY)))
# length_dict = {lengths[i]: str(i+200) for i in range(len(lengths))}
# BD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in BD_LENGTHONLY]
# BD2_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in BD2_LENGTHONLY]
# SD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in SD_LENGTHONLY]
# DD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in DD_LENGTHONLY]
# VD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in VD_LENGTHONLY]
# duration_all       = flatten(BD2_LENGTHONLY+BD_LENGTHONLY+DD_LENGTHONLY+SD_LENGTHONLY+VD_LENGTHONLY)
# duration_bach      = flatten(BD2_LENGTHONLY)
# duration_beethoven = flatten(BD_LENGTHONLY)
# duration_debussy   = flatten(DD_LENGTHONLY)
# duration_scarlatti = flatten(SD_LENGTHONLY)
# duration_victoria  = flatten(VD_LENGTHONLY)

# for l in [duration_all,duration_bach,duration_beethoven,duration_debussy,duration_scarlatti,duration_victoria]:
#     for key, value in Counter(l).most_common(10):
#         print(key, '&', round(100*value/len(l),2))
#     print('')

In [232]:
def crossvalidate_twofeaturesets(X_tuple1, X_tuple2, y, classifiers, vectorizer, range1, range2, K=5):    
    for clf in classifiers:
        clf.cm_sum = np.zeros([len(set(y)),len(set(y))], dtype='int16')
        clf.accuracies, clf.fones, clf.misclassified, clf.runningtime = [], [], [], []
        clf.fones_micro, clf.fones_macro = [], []
        clf.name = str(clf).split('(')[0]

    X1 = np.array([piece[0] for piece in X_tuple1])
    X2 = np.array([piece[0] for piece in X_tuple2])
    filenames = np.array([piece[1] for piece in X_tuple2])
    kf = KFold(n_splits=K, shuffle=True)
    for train_index, test_index in tqdm(kf.split(y), total=K, unit='fold', leave=False):
        y_train, y_test = y[train_index], y[test_index]
        X_train_new, X_test_new = X1[train_index], X1[test_index]
        vct1 = vectorizer.set_params(ngram_range=range1)
        X_train, X_test = X2[train_index], X2[test_index] 
        vct2 = vectorizer.set_params(ngram_range=range2)
   
        X_train_new_tfidf = vct1.fit_transform(X_train_new) # use two separate vectorizers for each feature set
        X_test_new_tfidf = sp.vstack([vct1.transform(np.array([piece])) for piece in X_test_new])
        X_train_tfidf = vct2.fit_transform(X_train)
        X_test_tfidf = sp.vstack([vct2.transform(np.array([piece])) for piece in X_test])
        
        X_train_tfidf = sp.hstack((X_train_tfidf, X_train_new_tfidf)) # Merge the two feature sets
        X_test_tfidf = sp.hstack((X_test_tfidf, X_test_new_tfidf))
        
        for clf in tqdm(classifiers, unit='classifier', leave=False):
            t = datetime.now()
            clf.fit(X_train_tfidf, y_train)
            y_pred = clf.predict(X_test_tfidf)
            clf.runningtime.append((datetime.now()-t).total_seconds())
            clf.cm_sum += confusion_matrix(y_test, y_pred)#, labels=range(1, len(composers)+1))
            clf.misclassified.append(test_index[np.where(y_test != y_pred)]) # http://stackoverflow.com/a/25570632
            clf.accuracies.append(accuracy_score(y_test, y_pred))
            clf.fones.append(f1_score(y_test, y_pred, average='weighted'))
            clf.fones_micro.append(f1_score(y_test, y_pred, average='micro'))
            clf.fones_macro.append(f1_score(y_test, y_pred, average='macro'))

    result = dict()
    for clf in classifiers:
        clf.misclassified = np.sort(np.hstack(clf.misclassified))
        result[clf.name] = [clf.cm_sum, clf.accuracies, clf.fones, clf.misclassified, filenames[clf.misclassified], clf.runningtime, clf.fones_micro, clf.fones_macro]
    return result

In [222]:
def benchmark_classifiers_twofeaturesets(composers1, composers2, range1, range2, classifiers, vectorizer, n=1, retrieve_title=True):
    misclassified_list = []
    print('duration n-gram range:', range1, 'chord n-gram range:', range2)
    X1, y = build_Xy(composers1, size=n)
    X2, y = build_Xy(composers2, size=n)
    cv_result = crossvalidate_twofeaturesets(X1, X2, y, classifiers, vectorizer, range1, range2)
    for clf, results in cv_result.items():
        print(clf)
        cm = results[0]
        print(cm)
        acc = results[1]
        fones = results[2]
        misclassified = results[3]
        misclassified_filenames = results[4]
        misclassified_list += list(misclassified_filenames)
#             print('misclassified',[(misclassified[i], misclassified_filenames[i]) for i in range(len(misclassified))])
        runningtime = results[5]
#         print('running time', np.sum(runningtime))
        fones_micro = results[6]
        fones_macro = results[7]
        print('F-measures')
        print(round(np.mean(fones_micro),4), '({})'.format(round(np.std(fones_micro, ddof=1),4)), '&', round(np.mean(fones_macro),4), '({})'.format(round(np.std(fones_macro, ddof=1),4)))
    print('-----')
    return misclassified_list

In [240]:
CLASSIFIERS = [
    svm.LinearSVC(penalty='l2', C=5, loss='hinge'),
    linear_model.LogisticRegression(penalty='l2', C=100, tol=1, multi_class='multinomial', solver='sag'),
    neighbors.KNeighborsClassifier(weights='distance'),
    naive_bayes.MultinomialNB(alpha=0.00001, fit_prior=False),
    neural_network.MLPClassifier(solver='lbfgs',hidden_layer_sizes=(10,)),
    RandomForestClassifier(n_estimators=500, random_state=42),
    GradientBoostingClassifier(n_estimators=500, max_leaf_nodes=4, max_depth=None, random_state=2,
                   min_samples_split=5),
              ]
VECTORIZER = TfidfVectorizer(sublinear_tf=True, lowercase=False, token_pattern=u"(?u)\\b\\w+\\b")

COMPOSERS1 = [BA_T, MO_T, BE_T, DE_T]

In [226]:
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERs, (1,2), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 2) chord n-gram range: (1, 2)


LinearSVC
[[39  6  5  0]
 [ 6 34  8  2]
 [ 9 12 26  3]
 [ 2  0  2 46]]
F-measures
0.725 (0.0755) & 0.7173 (0.0641)
LogisticRegression
[[29 11  7  3]
 [ 8 34  4  4]
 [12 17 18  3]
 [ 3  5  7 35]]
F-measures
0.58 (0.1111) & 0.5476 (0.1256)
KNeighborsClassifier
[[29 12  5  4]
 [ 9 30 10  1]
 [11 16 22  1]
 [ 2  4 15 29]]
F-measures
0.55 (0.0943) & 0.5329 (0.1066)
MultinomialNB
[[24  9 13  4]
 [ 5 18 24  3]
 [ 4  7 34  5]
 [ 1  0 11 38]]
F-measures
0.57 (0.1006) & 0.5556 (0.0937)
MLPClassifier
[[35  8  7  0]
 [ 6 33 10  1]
 [ 7 13 25  5]
 [ 3  0  7 40]]
F-measures
0.665 (0.0709) & 0.6461 (0.073)
RandomForestClassifier
[[27  7  8  8]
 [11 26  6  7]
 [ 8  9 26  7]
 [ 2  1  4 43]]
F-measures
0.61 (0.1101) & 0.5912 (0.1027)
GradientBoostingClassifier
[[30  9  9  2]
 [ 9 30  7  4]
 [10 12 26  2]
 [ 4  1  4 41]]
F-measures
0.635 (0.0851) & 0.6301 (0.0793)
-----


['Bach_001.mxl',
 'Bach_006.mxl',
 'Bach_007.mxl',
 'Bach_008.mxl',
 'Bach_012.mxl',
 'Bach_017.mxl',
 'Bach_020.mxl',
 'Bach_021.mxl',
 'Bach_031.mxl',
 'Bach_035.mxl',
 'Bach_045.mxl',
 'Mozart_051.mxl',
 'Mozart_052.mxl',
 'Mozart_054.mxl',
 'Mozart_057.mxl',
 'Mozart_062.mxl',
 'Mozart_065.mxl',
 'Mozart_066.mxl',
 'Mozart_075.mxl',
 'Mozart_079.mxl',
 'Mozart_081.mxl',
 'Mozart_082.mxl',
 'Mozart_086.mxl',
 'Mozart_091.mxl',
 'Mozart_094.mxl',
 'Mozart_097.mxl',
 'Mozart_099.mxl',
 'Beethoven_101.mxl',
 'Beethoven_103.mxl',
 'Beethoven_105.mxl',
 'Beethoven_107.mxl',
 'Beethoven_108.mxl',
 'Beethoven_110.mxl',
 'Beethoven_115.mxl',
 'Beethoven_116.mxl',
 'Beethoven_119.mxl',
 'Beethoven_123.mxl',
 'Beethoven_125.mxl',
 'Beethoven_126.mxl',
 'Beethoven_127.mxl',
 'Beethoven_129.mxl',
 'Beethoven_130.mxl',
 'Beethoven_132.mxl',
 'Beethoven_133.mxl',
 'Beethoven_135.mxl',
 'Beethoven_137.mxl',
 'Beethoven_138.mxl',
 'Beethoven_139.mxl',
 'Beethoven_140.mxl',
 'Beethoven_142.mxl',
 'B

In [ ]:
import scipy
from itertools import product
def benchmark_classifiers_twofeaturesets(composers1, composers2, range1, range2, classifier, vectorizer, n=1):
    #print('duration n-gram range:', range1, 'chord n-gram range:', range2)
    X1, y = build_Xy(composers1, size=n)
    X2, y = build_Xy(composers2, size=n)
    cv_result = crossvalidate_twofeaturesets(X1, X2, y, [classifier], vectorizer, range1, range2)
    results = list(cv_result.values())[0]
    fones_macro = results[7]
    return np.mean(fones_macro)
    
param_space = [
    {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant','adaptive'],
    },
    {'C': scipy.stats.expon(scale=100).rvs(4), 'gamma': scipy.stats.expon(scale=.1).rvs(4),
          'kernel': ['rbf'], 'class_weight':['balanced', None]},
    {'penalty': ['l1','l2'], 'C': 10.0 ** np.arange(-3, 4)},
    {'penalty': ['l1','l2'], 'C': 10.0 ** np.arange(-3, 4)},
    {'n_neighbors': [1, 2, 3, 4, 6, 8, 11, 20, 40], 'weights': ['distance', 'uniform'], 'p': [1, 2, 3]},
    {'n_estimators': [100, 300, 500, 800, 1200], 'max_depth': [5, 8, 15, 25, 30], 'min_samples_split': [2, 5, 10, 15, 100], 'min_samples_leaf': [1, 2, 5, 10],},
    {
        "n_estimators":[5,50,250,500],
        "max_depth":[1,3,5,7,9],
        "learning_rate":[0.01,0.1,1,10,100]
    } # https://www.datasciencelearner.com/gradient-boosting-hyperparameters-tuning/
][0]

model_cv = CLASSIFIERS[0]
scores = []
np.random.seed(42)

print(param_space.values())
configs = list(product(*param_space.values()))
for config in tqdm(configs, unit='config'): 
    print(config)
    print(dict(zip(param_space.keys(), config)))
    model_cv = neural_network.MLPClassifier(**dict(zip(param_space.keys(), config)))
    score = benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERs, (1,2), (1,2), model_cv, VECTORIZER)
    scores.append(score)

configs_scores = list(zip(configs, scores))
best_config, best_score = max(configs_scores, key=lambda x:x[1])
print('Best parameters found:\n', dict(zip(param_space.keys(), best_config)))

#scores = cross_val_score(model_cv, X, y)
#print(f'{name:^24} | score = {sum(scores)/len(scores):.3f}\n')

#print(' '.join(f'{score:.3f}' for score in scores))

print(f'''\n\nWe get a best average classification accuracy of {best_score:.3f}.
Compare this with the score for tf-idf on chord frequency later.''')

dict_values([[(50, 50, 50), (50, 100, 50), (100,)], ['tanh', 'relu'], ['sgd', 'adam'], [0.0001, 0.05], ['constant', 'adaptive']])


((50, 50, 50), 'tanh', 'sgd', 0.0001, 'constant')
{'hidden_layer_sizes': (50, 50, 50), 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.0001, 'learning_rate': 'constant'}


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

((50, 100, 50), 'relu', 'sgd', 0.05, 'adaptive')
{'hidden_layer_sizes': (50, 100, 50), 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.05, 'learning_rate': 'adaptive'}


((50, 100, 50), 'relu', 'adam', 0.0001, 'constant')
{'hidden_layer_sizes': (50, 100, 50), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'constant'}


((50, 100, 50), 'relu', 'adam', 0.0001, 'adaptive')
{'hidden_layer_sizes': (50, 100, 50), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.0001, 'learning_rate': 'adaptive'}


((50, 100, 50), 'relu', 'adam', 0.05, 'constant')
{'hidden_layer_sizes': (50, 100, 50), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.05, 'learning_rate': 'constant'}


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [216]:
for indices in combinations(range(len(composers)),2):
    print('composer indices', [i for i in indices]) 
    benchmark_classifiers_twofeaturesets([COMPOSERS1[i] for i in indices],[COMPOSERs[i] for i in indices],(1,1),(1,2),CLASSIFIERS,VECTORIZER)

composer indices [0, 1]
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[42  8]
 [ 8 42]]
F-measures
0.84 (0.0516) & 0.8226 (0.0441)
LogisticRegression
[[30 20]
 [20 30]]
F-measures
0.6 (0.1491) & 0.562 (0.1863)
KNeighborsClassifier
[[34 16]
 [16 34]]
F-measures
0.68 (0.1033) & 0.6564 (0.125)
MLPClassifier
[[42  8]
 [ 7 43]]
F-measures
0.85 (0.085) & 0.8334 (0.0874)
RandomForestClassifier
[[35 15]
 [20 30]]
F-measures
0.65 (0.165) & 0.624 (0.1831)
GradientBoostingClassifier
[[37 13]
 [14 36]]
F-measures
0.73 (0.1418) & 0.7108 (0.1491)
-----
composer indices [0, 2]
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[42  8]
 [13 37]]
F-measures
0.79 (0.0994) & 0.7745 (0.0931)
LogisticRegression
[[37 13]
 [22 28]]
F-measures
0.65 (0.1354) & 0.589 (0.165)
KNeighborsClassifier
[[38 12]
 [13 37]]
F-measures
0.75 (0.178) & 0.7341 (0.1829)
MLPClassifier
[[39 11]
 [ 9 41]]
F-measures
0.8 (0.1054) & 0.7825 (0.1041)
RandomForestClassifier
[[38 12]
 [13 37]]
F-

In [217]:
# Identify the often-misclassified files, using both feature sets
# do the experiment 100 times with the best classifier, SVM
# then find pieces that are misclassified more than 50% of the time
COMPOSERS1 = [BA_T, MO_T, BE_T, DE_T]
#COMPOSERS2 = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]
CLASSIFIERS = [svm.LinearSVC(penalty='l2', C=5, loss='hinge')] # 
appendix = []
for i in trange(100):
    appendix += benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERs, (1,1), (1,2), CLASSIFIERS, VECTORIZER)
Counter(appendix).most_common()

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[40  5  5  0]
 [ 4 39  5  2]
 [ 9 11 27  3]
 [ 2  0  1 47]]
F-measures
0.765 (0.0851) & 0.735 (0.082)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[38  7  5  0]
 [ 6 35  7  2]
 [ 9 13 25  3]
 [ 1  0  1 48]]
F-measures
0.73 (0.0587) & 0.6993 (0.0911)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[40  5  5  0]
 [ 6 35  7  2]
 [ 9 12 27  2]
 [ 2  0  1 47]]
F-measures
0.745 (0.0864) & 0.7118 (0.1097)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[40  6  4  0]
 [ 5 36  7  2]
 [ 7 12 29  2]
 [ 1  0  1 48]]
F-measures
0.765 (0.058) & 0.7463 (0.0574)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[39  6  5  0]
 [ 7 36  5  2]
 [10 12 25  3]
 [ 1  0  2 47]]
F-measures
0.735 (0.1029) & 0.7175 (0.112)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[35  9  6  0]
 [ 6 36  6  2]
 [10 12 26  2]
 [ 

KeyboardInterrupt: 

In [ ]:
X, y = build_Xy(COMPOSERs, size=1)
print(X)
y